In [1]:
import tensorflow as tf
import gym


import os,cv2, datetime,numpy as np
from tensorflow.keras import callbacks
import dataManager
from nets import simpleDense as net

In [2]:
nn = net.NeuralNetwork()
model,_ = nn.getModel((80*80),(1))

model could not be loaded: [Errno 2] No such file or directory: 'saveData/SimpleFNN/'
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               1280200   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 1,280,401
Trainable params: 1,280,401
Non-trainable params: 0
_________________________________________________________________


In [3]:
# gym initialization
env = dataManager.SingleGym("Pong-v0",True)
observation = env.reset()

# Macros
UP_ACTION = 2
DOWN_ACTION = 3

In [4]:
today = datetime.datetime.today()
time = "%s%04d-%02d-%02d-%02d-%02d-%02d/" % (nn.getModelFolderPath(),today.year,today.month,today.day,today.hour,today.minute,today.second)
os.makedirs("%s/figs"%(time))

with open('%s/architecture.txt'%(time),'w') as fh:
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
    
csvLogger = callbacks.CSVLogger(time+"log.csv", separator=',', append=False)
tensorboardCallback = callbacks.TensorBoard(log_dir=time)#TODO: load test data in advance and use histogram_freq=1 here
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(time)
file_writer.set_as_default()


callbacks =  [callbacks.ModelCheckpoint(time+"{epoch:04d}.hdf5",
monitor='val_loss',verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=5),
csvLogger,tensorboardCallback]

In [5]:
# Hyperparameters
gamma = 0.99

# initialization of variables used in the main loop
x_train, y_train, rewards = [],[],[]
reward_sum = 0
episode = 0
running_reward = 0

while True:
    
    # forward the policy network and sample action according to the proba distribution
    upProbability = model.predict(np.array([observation.squeeze(axis=2).ravel()]))[0]
    if np.random.uniform() < upProbability:
        action = UP_ACTION
    else:
        action = DOWN_ACTION
    y = 1 if action == 2 else 0 # 0 and 1 are our labels

    # log the input and label to train later
    x_train.append([observation.squeeze(axis=2).ravel()])
    y_train.append(y)

    # do one step in our environment
    observation, reward, done, info = env.step(action)
    rewards.append(reward)
    reward_sum += reward
    
    env.env.render()
    # end of an episode
    if done:
        print('At the end of episode', episode, 'the total reward was :', reward_sum)
        
        # training
        model.fit(x=np.vstack(x_train), y=np.vstack(y_train), verbose=1, callbacks=callbacks, sample_weight=dataManager.calculateRewards(rewards, gamma),initial_epoch = episode, epochs = episode+1)
        
        running_reward = running_reward * 0.99 + reward_sum * 0.01
        #Log Running Reward
        tf.summary.scalar("Running Reward", running_reward,step=episode)
        tf.summary.scalar("Reward Sum", reward_sum,step=episode)
        
        
        # increment episode number
        episode += 1
        
        # Reinitialization
        x_train, y_train, rewards = [],[],[]
        observation = env.reset()
        reward_sum = 0
        prev_input = None

At the end of episode 0 the total reward was : -21.0
45/45 [==============================] - 1s 8ms/step - loss: -7.6337e-04 - accuracy: 0.4908
At the end of episode 1 the total reward was : -21.0
Epoch 2/2
52/52 [==============================] - 0s 6ms/step - loss: -0.0098 - accuracy: 0.5178
At the end of episode 2 the total reward was : -18.0
Epoch 3/3
52/52 [==============================] - 0s 6ms/step - loss: 0.0051 - accuracy: 0.5801
At the end of episode 3 the total reward was : -21.0
Epoch 4/4
43/43 [==============================] - 0s 7ms/step - loss: -0.0093 - accuracy: 0.5937
At the end of episode 4 the total reward was : -21.0
Epoch 5/5
33/33 [==============================] - 0s 9ms/step - loss: 0.0101 - accuracy: 0.6277

Epoch 00005: saving model to saveData/SimpleFNN/2021-05-31-22-05-20/0005.hdf5
At the end of episode 5 the total reward was : -21.0
Epoch 6/6
37/37 [==============================] - 0s 8ms/step - loss: 0.0036 - accuracy: 0.6071
At the end of episode 6 

KeyboardInterrupt: 